In [19]:
import java.io.File;
import java.util.Scanner;
import java.lang.Float;

## Assignment 3 (Group)

Für diese Aufgabe müssen Sie zuerst die Datei `data.txt` aus MyLearn in Ihrer Jupyter-Umgebung ablegen; diese muss im gleichen Verzeichnis wie dieses Notebook untergebracht werden.

Mit den folgenden Zeilen Programmcode wird zuerst eine Klasse `Person` erstellt. Eine Person wird durch einen Namen, ihr Geschlecht, und eine Liste, in der die Kinder dieser Person gespeichert werden (wieder als Person-Objekte), beschrieben.

In [20]:
class Person {
    String name;
    String sex;
    List<Person> children;
    
    public Person(String name, String sex) {
        this.name = name;
        this.sex = sex;
        this.children = new ArrayList<Person>();
    }
}

Hier werden nun aus der Date `data.txt` eine Reihe von Personen und deren Beziehung (`parentOf`) ausgelesen. Diese werden in der Map `allPersons` gespeichert:

$ allPersons := \{jan \rightarrow \{name \rightarrow jan,  sex \rightarrow male, children \rightarrow \{...\},  ...\} $


In [21]:
Map<String, Person> allPersons = new HashMap<String, Person>();

File file = new File("data.txt");
Scanner sc = new Scanner(file);
String name;
String newLine;
String[] names;

/* In a first iteration, we add all people */
while (sc.hasNextLine()) {
    newLine = sc.nextLine();
    if (newLine.indexOf("female") == 0) {
        name = newLine.substring((newLine.indexOf("(")+1), newLine.indexOf(")"));
        allPersons.put(name, new Person(name, "female"));
    } else if (newLine.indexOf("male") == 0) {
        name = newLine.substring((newLine.indexOf("(")+1), newLine.indexOf(")"));
        allPersons.put(name, new Person(name, "male"));
    }
}

file = new File("data.txt");
sc = new Scanner(file);

/* In another iteration, we add all children */
while (sc.hasNextLine()) {
    newLine = sc.nextLine();
    if (newLine.indexOf("parentOf") == 0) {
        newLine = newLine.substring((newLine.indexOf("(")+1), newLine.indexOf(")"));
        names = newLine.split(",");
        allPersons.get(names[0]).children.add(allPersons.get(names[1]));
    }
}

In [22]:
/* Check the number of recorded persons */
assert allPersons.size() == 2000;

In [23]:
/* access a single person in allPersons via the name as key */
Person p1 = allPersons.get("jan");

System.out.println("Name: " + p1.name);
System.out.println("Sex: " + p1.sex);
System.out.println("Children:");
for (Person p : p1.children) {
    System.out.println(p.name);
}

Name: jan
Sex: male
Children:
marko
anna15


## Aufgabe 1

* Berechnen Sie die maximale und die durchschnittliche Anzahl an Kindern in dieser Personenliste.
* Vervollständigen Sie `describe`. Diese Funktion retourniert ein Array mit zwei `float`-Einträgen. 
    - Feldeintrag 0 enthält die maximale Anzahl an Kindern über alle Personen.
    - Feldeintrag 1 enthält die durchschnittliche Anzahl an Kindern über alle Personen.

In [24]:
float[] describe(Map<String, Person> m) {
    // YOUR CODE HERE
    
    float maxKinder = 0;
    float anzahlKinder = 0;
    float anzahlPersonen = 0;
    
    for(Map.Entry<String, Person> aktuellerWert : m.entrySet()) {

        Person personValue = aktuellerWert.getValue();
        
        // Maximale Anzahl Kinder
        if (personValue.children.size() > maxKinder) {
            maxKinder = personValue.children.size();
        }
        
        anzahlKinder = anzahlKinder + personValue.children.size();
        anzahlPersonen++;
    }
    
    float durchAnzahlKinder = anzahlKinder / anzahlPersonen;
    
    return new float[] {maxKinder, durchAnzahlKinder};       
}

// Testausgabe
float[] rueckgabe = describe(allPersons);
System.out.println("maximale Anzahl an Kinder: " + rueckgabe[0]);
System.out.println("durchschnittliche Anzahl an Kinder: " + rueckgabe[1]);

maximale Anzahl an Kinder: 5.0
durchschnittliche Anzahl an Kinder: 1.354


In [25]:
// Do not edit this cell.

## Aufgabe 2

* Wie viele Personen ohne Vorfahren gibt es (*Wurzelknoten*)?
* Vervollständigen Sie `findRoots`. Die Funktion retourniert eine Liste von allen Personen, die keine Vorfahren haben (*Personenobjekte*). 

In [26]:
List<Person> findRoots(Map<String, Person> m) {
    List<String> children = new ArrayList<String>();
    
    // Alle Kinder in Liste geben
    for(Map.Entry<String, Person> aktuellerWert : m.entrySet()) {

        Person personValue = aktuellerWert.getValue();
        
        for (Person p : personValue.children) {
            children.add(p.name);
        }
    }
    
    List<Person> rootPersons = new ArrayList<Person>();
    
    // Alle Personen durchgehen und vergleichen ob sie in Liste von allen Kindern sind, wenn nicht, sind sie Roots
    for(Map.Entry<String, Person> aktuellerWert : m.entrySet()) {

        Person personValue = aktuellerWert.getValue();
        Boolean root = true;
        
        for (String child : children) {
            if (child.equals(personValue.name)) {
                root = false;
                break;
            } 
        }
        
        if (root) {
            Person newRoot = new Person(personValue.name, personValue.sex);
            newRoot.children = personValue.children;
            
            rootPersons.add(newRoot);
        }
    }
    
    return rootPersons;
}

// Testausgabe
List<Person> roots = findRoots(allPersons);

System.out.println("Es gibt "+ roots.size() + " Personen ohne Vorfahren.");

Es gibt 646 Personen ohne Vorfahren.


In [27]:
/* DO NOT ALTER OR DELETE THIS CELL! */

## Aufgabe 3

* Bestimmen Sie dazu den längsten Generationenpfad, und geben Sie ihn aus.
* Vervollständigen Sie `findSinglePath` und `findLongestPath`.
* `findSinglePath` soll die Anzahl der nachkommenden Generation einer Person ausgeben. 
  * Wenn die Person keine Nachkommen hat, wird 1 retourniert.
  * Wenn die Person Nachkommen hat, wird > 1 retourniert, d.h. für jede Generation um 1 erhöht. (d.h.: keine Kinder: 1; Kinder: 2; Enkelkinder: 3 usw.)
* `findLongestPath` gibt die Länge der längsten Generationenkette aus. 
        

In [28]:
int findSinglePath(Person p) {
    // YOUR CODE HERE   
    
    int anzahl = 1;
    int maxGenerationLevel = 0;
    int temporalGenerationenLevel = 0;
    
    if (p.children.size() > 0)
    {
        for (Person child : p.children) {
            temporalGenerationenLevel = anzahl + findSinglePath(child);
            
            if (temporalGenerationenLevel > maxGenerationLevel) {
                maxGenerationLevel = temporalGenerationenLevel;
            }
        }
        
        anzahl = maxGenerationLevel;
    }
        
    return anzahl;
}

int findLongestPath(Map<String, Person> m) {
    // YOUR CODE HERE
    int maxGeneration = 0;
    for(Map.Entry<String, Person> aktuellerWert : m.entrySet()) {     
        int aktGeneration = findSinglePath(aktuellerWert.getValue());
        
        if (aktGeneration > maxGeneration)
        {
            maxGeneration = aktGeneration;
        }
    }    
    
    return maxGeneration;
}

In [29]:
/* DO NOT ALTER OR DELETE THIS CELL! */
assert findSinglePath(p1) == 2;

## Aufgabe 4

* Wie oft kommen Personen mit einem bestimmten Vornamen (z.B.: `"stefan"`) über die diversen Eltern-Kind-Beziehungen und Generationen vor?
* Komplettieren Sie die untenstehende Funktion `count`, um einen assoziiativen Array (Map) zu befüllen, welche für die eindeutigen Vornamen die Zahl dieser Vorkommnisse in `allPersons` führt.
* Beachten Sie, dass die Vornamen numerische Zusätze (Suffizes) aufweisen (z.B.: `"stefan3"`), welche Sie bei der Zählung ignorieren sollen (`"stefan3"` zählt als Vorkommnis von `"stefan"`);
* Beachten Sie dafür, dass die Baumstruktur (wenn komplett und korrekt aufgebaut) wie folgt aussieht:
  * `(root)` steht für einen erdachten, gemeinsamen Wurzelnoten über die die "Roots", wie oben definiert.
  * Der Zahlenwert vor jeder Zeile steht für die Pfadlänge vom gemeinsamen Wurzelknoten (`(root)`) zum jeweiligen Knoten.
  * Ein und derselbe Name mit numerischem Zusatz kann mehrfach vorkommen (nicht nur als Eltern-, sondern auch als Kindteil), in Darstellung unterschiedlichster Eltern-Kind-Beziehungsarten;

```
0: (root)
# ...
1:   hannah18
2:     fabian17
3:       tobias16
4:         matthias10
5:           luca3
6:             tobias8
7:               oliver9
6:             sofia3
7:               daniel6
8:                 rene12
8:                 lena13
7:               philipp12
6:             hannah9
6:             julia12
6:             valentina6
7:               leo20
4:         anna18
4:         valentin17
# ...
```

In [30]:
void count(Person p, Map<String, Integer> counts) {
    
    List<Person> rootList = findRoots(allPersons); 
    String vergleichsName = removeDigits(p.name);
    counts.put(vergleichsName, 0);
    
     for(Person r : rootList) {
         IsIncluded(vergleichsName, r, counts);
     }
}


public static String removeDigits (String dgt) {
     return dgt.replaceAll("\\d","");   
}


void IsIncluded(String name, Person p, Map <String, Integer> counts) {
    if(name.equals(removeDigits(p.name))) { 
        counts.put(name, counts.get(name) + 1);
    }
    
    for (Person child : p.children) {
        IsIncluded(name, child, counts);
    } 
}

In [31]:
/* DO NOT ALTER OR DELETE THIS CELL! */
Map<String, Integer> m = new HashMap<String, Integer>();
for (Person person : findRoots(allPersons)) {
    count(person, m);
}
assert m.get("stefan") == 128;
assert m.get("leo") == 129;


<h1><span style="color:blue">Tagebuch: </span> </h1>

<b>Vorgehensweise:</b><br> 
Erstellen einer WhatsApp Gruppe und eines Teams Channels für die Besprechungen. <br>
Besprechungen am 14.05. ,17.05. und 18.05. (Bis zum Ersten Meeting hat sich jedes Mitglied der Gruppe bereits die Aufgaben angesehen oder versucht zu lösen). <br> 

<b>Aufgabe 1: </b><br>
<i>Schwierigkeiten:</i> 
* Wie iteriert man durch die HashMap? 
* Parameterrückgabe 

`Gelöst am: 14.05.`

<b>Aufgabe 2:</b> <br>
<i>Schwierigkeiten:</i> 
* Konzept finden
* Children untereinander vergleichen 

`Gelöst am: 14.05.`
    
<b>Aufgabe 3:</b> <br>
<i>Schwierigkeiten:</i> 
 * Sprung in die nächste Generation automatisieren -> Rekursion einbauen 

`Gelöst am: 14.05.` 
    
<b>Aufgabe 4:</b><br>
<i>Schwierigkeiten:</i> 
 * Strings von Zahlen bereinigen -> versuch mit "replaceAll()", hat ohne eigene Funktion nicht funktioniert, implementierung mit charAt und Character.isDigit() nicht so schlank
 * "Alles in eines" Code nicht gut machbar -> Aufteilen des Codes in drei Teile 
 
`Gelöst am: 17.05.`

<b>Linkverzeichnis<b><br> 

 *  `Methoden zur Bereinigung der Strings:` prashant_srivastava, geeksforgeeks.org, 2020, https://www.geeksforgeeks.org/different-ways-to-remove-all-the-digits-from-string-in-java/ (Zugriff am 17.05.)

    